In [ ]:
import numpy as np
import pandas as pd
import keras

from timeit import default_timer
from sklearn.model_selection import train_test_split
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
import tensorflow as tf
from google.colab import userdata
import os

!pip install kaggle


In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
!kaggle datasets download -d radcliffe/3-million-sudoku-puzzles-with-ratings
!unzip 3-million-sudoku-puzzles-with-ratings.zip
''' !pip install nvidia-smi
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
!top '''

3-million-sudoku-puzzles-with-ratings.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  3-million-sudoku-puzzles-with-ratings.zip
replace sudoku-3m.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

" !pip install nvidia-smi\ngpu_info = !nvidia-smi\ngpu_info = '\n'.join(gpu_info)\nif gpu_info.find('failed') >= 0:\n  print('Not connected to a GPU')\nelse:\n  print(gpu_info)\n!top "

In [ ]:
# Create a model
def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))

    return model

In [ ]:
# Read the dataset
data = pd.read_csv("sudoku-3m.csv")

KeyboardInterrupt: 

In [ ]:
print(data.columns)


Index(['id', 'puzzle', 'solution', 'clues', 'difficulty'], dtype='object')


In [ ]:


def preprocess_data(data):
    # Replace periods with zeros in puzzle strings
    data['puzzle'] = data['puzzle'].str.replace('.', '0')

    # Extract Sudoku grid values (first 81 characters of the puzzle string)
    X = np.array([list(map(int, puzzle[:81])) for puzzle in data['puzzle']])
    y = data['solution'].values  # No need to adjust targets here

    # Reshape X to have dimensions (num_samples, 9, 9, 1)
    X = X.reshape(-1, 9, 9, 1)

    # Normalize input
    X = X / 9.0  # Sudoku values range from 1 to 9

    return X, y


# Split dataset into train and test sets
X, y = preprocess_data(data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
import tensorflow as tf

# One-hot encode the labels
y_train_encoded = tf.one_hot(y_train, depth=9)
y_test_encoded = tf.one_hot(y_test, depth=9)

# Create the model
def get_model():
    model = keras.models.Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(9, 9, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1, 1), activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(81 * 9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    return model

# Train the model
model = get_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_encoded, batch_size=64, epochs=10, validation_data=(X_test, y_test_encoded))

# Define the solver function
def solve_sudoku(puzzle):
    # Preprocess puzzle
    puzzle = np.array(puzzle).reshape(9, 9, 1) / 9.0  # Normalize input

    # Use the model to predict
    solution_probabilities = model.predict(np.array([puzzle]))[0]

    # Convert probabilities to values
    solution = np.argmax(solution_probabilities, axis=1).reshape(9, 9) + 1  # Adjust back to 1-9 range

    return solution

# Define main function to test solver on a single Sudoku puzzle
def main():
    # Example Sudoku puzzle
    puzzle = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    # Solve Sudoku puzzle
    solution = solve_sudoku(puzzle)

    # Print solved Sudoku
    print("Solved Sudoku:")
    print(solution)

if __name__ == "__main__":
    main()


<ipython-input-20-56069b8caf0c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['puzzle'] = data['puzzle'].str.replace('.', '0')


OverflowError: Python int too large to convert to C long